Packages to Install

In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pytz

Download Price Data

In [5]:
data = yf.download('GC=F', interval='5m', period='7d')
print(data.head)

[*********************100%***********************]  1 of 1 completed

<bound method NDFrame.head of Price                            Close         High          Low         Open  \
Ticker                            GC=F         GC=F         GC=F         GC=F   
Datetime                                                                        
2025-03-28 04:00:00+00:00  3084.800049  3085.500000  3084.399902  3084.899902   
2025-03-28 04:05:00+00:00  3083.600098  3083.899902  3083.600098  3083.899902   
2025-03-28 04:15:00+00:00  3084.600098  3084.600098  3083.399902  3083.399902   
2025-03-28 04:20:00+00:00  3084.199951  3084.399902  3084.100098  3084.399902   
2025-03-28 04:25:00+00:00  3084.600098  3084.600098  3084.300049  3084.300049   
...                                ...          ...          ...          ...   
2025-04-04 12:35:00+00:00  3099.800049  3115.100098  3074.500000  3112.199951   
2025-04-04 12:40:00+00:00  3108.699951  3108.800049  3065.500000  3099.399902   
2025-04-04 12:45:00+00:00  3098.600098  3110.000000  3071.000000  3107.800049  

**SGT Timezone**

In [8]:
data = data.reset_index()
data['Datetime'] = data['Datetime'].dt.tz_convert('Asia/Singapore')
print(data['Datetime'].head())

0   2025-03-28 12:00:00+08:00
1   2025-03-28 12:05:00+08:00
2   2025-03-28 12:15:00+08:00
3   2025-03-28 12:20:00+08:00
4   2025-03-28 12:25:00+08:00
Name: Datetime, dtype: datetime64[ns, Asia/Singapore]


**Define Asia High/Lows**

In [15]:
def get_asia_high_low(df):
    today = datetime.now(pytz.timezone('Asia/Singapore'))
    yesterday = today - timedelta(days=1)
    asia_start = yesterday.replace(hour=23, minute=0, second=0, microsecond=0)
    asia_end = asia_start + timedelta(hours=7)
    asia_df = df[(df['Datetime'] >= asia_start) & (df['Datetime'] <= asia_end)]
    if asia_df.empty:
        print("No data found for Asia session. Check data range or timezone.")
        return None, None
    asia_high = asia_df['High'].max()
    asia_low = asia_df['Low'].min()
    return asia_high, asia_low
asia_high, asia_low = get_asia_high_low(data)
if asia_high is not None and asia_low is not None:
    asia_high_val = asia_high.item() if hasattr(asia_high, 'item') else asia_high
    asia_low_val = asia_low.item() if hasattr(asia_low, 'item') else asia_low

    if not pd.isna(asia_high_val) and not pd.isna(asia_low_val):
        print("Asia Session High:", asia_high_val)
        print("Asia Session Low:", asia_low_val)
    else:
        print("Asia session data exists, but high/low are NaN.")
else:
    print("Asia session data was not found.")


Asia Session High: 3152.800048828125
Asia Session Low: 3099.0


**London Sweep**

In [31]:
def check_london_sweep(df, asia_high, asia_low):
    today = datetime.now(pytz.timezone('Asia/Singapore')).replace(hour=14, minute=0, second=0, microsecond=0)
    london_start = today
    london_end = london_start + timedelta(hours=4)
    london_df = df[(df['Datetime'] >= london_start) & (df['Datetime'] <= london_end)]

    if london_df.empty:
        print("No data found for London session.")
        return None
    sweep_high = (london_df['High'] > asia_high).any()
    sweep_low = (london_df['Low'] < asia_low).any()
    return sweep_high, sweep_low, london_df
sweep_high, sweep_low, london_df = check_london_sweep(data, asia_high_val, asia_low_val)
sweep_high = sweep_high.item() if hasattr(sweep_high, 'item') else sweep_high
sweep_low = sweep_low.item() if hasattr(sweep_low, 'item') else sweep_low

if sweep_high:
    print("London session **swept Asia HIGH** – potential SHORT setup.")
if sweep_low:
    print("London session **swept Asia LOW** – potential LONG setup.")
if not sweep_high and not sweep_low:
    print("No sweeps during London session.")

London session **swept Asia LOW** – potential LONG setup.


**Bullish/Bearish Engulfing Candle Detection**

In [36]:
def detect_engulfing(df):
    engulfing_signals = []

    for i in range(1, len(df)):
        prev = df.iloc[i - 1]
        curr = df.iloc[i]
        prev_open = prev['Open'].item() if hasattr(prev['Open'], 'item') else prev['Open']
        prev_close = prev['Close'].item() if hasattr(prev['Close'], 'item') else prev['Close']
        curr_open = curr['Open'].item() if hasattr(curr['Open'], 'item') else curr['Open']
        curr_close = curr['Close'].item() if hasattr(curr['Close'], 'item') else curr['Close']
        if (prev_close < prev_open) and (curr_close > curr_open) and \
           (curr_open < prev_close) and (curr_close > prev_open):
            engulfing_signals.append(('bullish', curr['Datetime'].iloc[0]))
        elif (prev_close > prev_open) and (curr_close < curr_open) and \
             (curr_open > prev_close) and (curr_close < prev_open):
            engulfing_signals.append(('bearish', curr['Datetime'].iloc[0]))

    return engulfing_signals

engulfing_signals = detect_engulfing(london_df)

for signal in engulfing_signals:
    print(f"Engulfing Detected: {signal[0].upper()} at {signal[1]}")


Engulfing Detected: BULLISH at 2025-04-04 15:05:00+08:00
Engulfing Detected: BULLISH at 2025-04-04 15:45:00+08:00
Engulfing Detected: BEARISH at 2025-04-04 17:55:00+08:00


**Alert System**

In [41]:
import requests
BOT_API_TOKEN = '7724318587:AAF2pB62HMrrjA2oRgg4r59j2UNfwA4WFHM'
CHAT_ID = '5791808659'

def send_telegram_message(message):
    url = f"https://api.telegram.org/bot{BOT_API_TOKEN}/sendMessage"
    payload = {
        'chat_id': CHAT_ID,
        'text': message
    }
    response = requests.post(url, data=payload)
    if response.status_code == 200:
        print(f"Message sent to {CHAT_ID}")
    else:
        print(f"Failed to send message: {response.text}")

def detect_engulfing(df):
    engulfing_signals = []

    for i in range(1, len(df)):
        prev = df.iloc[i - 1]
        curr = df.iloc[i]

        prev_open = prev['Open'].item() if hasattr(prev['Open'], 'item') else prev['Open']
        prev_close = prev['Close'].item() if hasattr(prev['Close'], 'item') else prev['Close']
        curr_open = curr['Open'].item() if hasattr(curr['Open'], 'item') else curr['Open']
        curr_close = curr['Close'].item() if hasattr(curr['Close'], 'item') else curr['Close']
        if (prev_close < prev_open) and (curr_close > curr_open) and \
           (curr_open < prev_close) and (curr_close > prev_open):
            engulfing_signals.append(('bullish', curr['Datetime']))
            send_telegram_message(f"Engulfing Detected: BULLISH at {curr['Datetime']}")
        elif (prev_close > prev_open) and (curr_close < curr_open) and \
             (curr_open > prev_close) and (curr_close < prev_open):
            engulfing_signals.append(('bearish', curr['Datetime']))
            send_telegram_message(f"Engulfing Detected: BEARISH at {curr['Datetime']}")

    return engulfing_signals
engulfing_signals = detect_engulfing(london_df)
for signal in engulfing_signals:
    print(f"Engulfing Detected: {signal[0].upper()} at {signal[1]}")


Message sent to 5791808659
Message sent to 5791808659
Message sent to 5791808659
Engulfing Detected: BULLISH at Ticker
   2025-04-04 15:05:00+08:00
Name: 1413, dtype: datetime64[ns, Asia/Singapore]
Engulfing Detected: BULLISH at Ticker
   2025-04-04 15:45:00+08:00
Name: 1420, dtype: datetime64[ns, Asia/Singapore]
Engulfing Detected: BEARISH at Ticker
   2025-04-04 17:55:00+08:00
Name: 1446, dtype: datetime64[ns, Asia/Singapore]
